### 0

In [966]:
import pandas as pd
from sqlalchemy import create_engine

# Загружаем Excel файл
file_path = 'Копия hh_аналитик_таблицы_01 (2) (1).xlsx'
xls = pd.ExcelFile(file_path)

# Создаем подключение к SQLite
engine = create_engine('sqlite:///hh_аналитик.db')

# Преобразуем каждый лист в таблицу SQLite
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    df.to_sql(sheet_name, con=engine, if_exists='replace', index=False)

In [967]:
# Выполняем SQL-запрос для получения списка таблиц
with engine.connect() as connection:
    result = connection.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = result.fetchall()

# Выводим список таблиц
for table in tables:
    print(table[0])

description_tables
vacancy
resume
area
currency


In [950]:
query_vacancy = "SELECT * FROM vacancy"
query_resume = "SELECT * FROM resume"
query_area = "SELECT * FROM area"

In [951]:
with engine.connect() as connection:
    df_vacancy = pd.read_sql(query_vacancy, connection)
    df_resume = pd.read_sql(query_resume, connection)
    df_area = pd.read_sql(query_area, connection)

    
df_vacancy.head()

,vacancy_id,name,work_schedule,disabled,area_id,creation_time,archived
0,1,Курьер,Гибкий график,1,1,2020-01-01 14:02:00,0
1,2,Аналитик,Полный день,0,2,2021-02-10 13:13:00,1
2,3,Маркетолог,Удаленная работа,1,3,2022-02-01 11:25:00,0
3,4,Разработчик,Полный день,0,1,3.15.22 09:45,0
4,5,Личный водитель,Гибкий график,1,3,5/20/21 08:30,0


In [952]:
df_resume.head()

,resume_id,disabled,is_finished,area_id,compensation,currency,birth_day,role_id_list
0,1,0,1,1,300.0,EUR,2008-05-06 00:00:00.000000,"[5, 12, 18]"
1,2,0,1,2,100000.0,RUR,2007-05-06 00:00:00.000000,"[3, 7]"
2,3,0,3,1,1500.0,USD,2006-12-12 00:00:00.000000,[14]
3,4,0,1,1,NaN,EUR,1989-05-06 00:00:00.000000,"[8, 15]"
4,5,1,2,2,50000.0,RUR,1985-02-09 00:00:00.000000,"[1, 9]"


In [953]:
df_area

,area_id,area_name,region_name,country_name
0,1,Москва,Москва,Россия
1,2,Санкт-Петербург,Санкт-Петербург,Россия
2,3,Новосибирск,Новосибирская область,Россия


### 1

In [954]:
query = """
SELECT 
    name, 
    work_schedule, 
    disabled, 
    creation_time,
    strftime('%Y', creation_time) AS year,
    strftime('%m', creation_time) AS month,
    area_name
    FROM vacancy
JOIN area USING (area_id)
WHERE
    creation_time BETWEEN '2020-01-01' AND '2023-01-01' AND
    LOWER(name) LIKE '%Водитель%'
ORDER BY creation_time
"""

In [955]:
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    
df

,name,work_schedule,disabled,creation_time,year,month,area_name
0,Водитель грузоперевозок,Удаленная работа,0,2021-05-12 14:10:00,2021,05,Санкт-Петербург
1,Водитель-погрузчик,Гибкий график,1,2021-07-07 09:20:00,2021,07,Москва


### 2

In [956]:
query = """
SELECT 
    region_name,
    COUNT(*) count_vacancy
    FROM vacancy
JOIN area USING (area_id)
WHERE work_schedule = 'Удаленная работа'
GROUP BY region_name
"""

In [957]:
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    
df

,region_name,count_vacancy
0,Москва,1
1,Новосибирская область,2
2,Санкт-Петербург,4


In [958]:
query = """
SELECT 
    region_name,
    COUNT(*) count_vacancy
    FROM vacancy
JOIN area USING (area_id)
GROUP BY region_name
"""

In [959]:
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    
df

,region_name,count_vacancy
0,Москва,11
1,Новосибирская область,8
2,Санкт-Петербург,11


In [960]:
query = """
SELECT
    region_name,
    COUNT(*) AS total_vacancies,
    SUM(CASE WHEN work_schedule = 'Удаленная работа' THEN 1 ELSE 0 END) AS remote_vacancies,
    SUM(CASE WHEN work_schedule = 'Удаленная работа' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS remote_vacancy_ratio
FROM
    vacancy
JOIN
    area USING (area_id)
--WHERE
--    country_name = 'Россия' AND
--    creation_time BETWEEN '2021-01-01' AND '2023-01-01' AND
--    disabled = FALSE AND
--    archived = FALSE
GROUP BY
    region_name
ORDER BY 
    remote_vacancy_ratio DESC
LIMIT
    10
"""

In [961]:
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    
df

,region_name,total_vacancies,remote_vacancies,remote_vacancy_ratio
0,Санкт-Петербург,11,4,0.363636
1,Новосибирская область,8,2,0.250000
2,Москва,11,1,0.090909


In [962]:
query = """

SELECT
    region_name,
    COUNT(*) AS total_vacancies,
    SUM(CASE WHEN work_schedule = 'Удаленная работа' THEN 1 ELSE 0 END) AS remote_vacancies,
    SUM(CASE WHEN work_schedule = 'Удаленная работа' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS remote_vacancy_ratio
FROM
    vacancy
JOIN
    area USING (area_id)
GROUP BY
    region_name
ORDER BY 
    remote_vacancy_ratio DESC

"""

In [985]:
query = """
WITH total_remote_vacancies AS (
    SELECT
        SUM(CASE WHEN work_schedule = 'Удаленная работа' THEN 1 ELSE 0 END) AS total_remote
    FROM vacancy
)
SELECT
    region_name,
    COUNT(*) AS total_vacancies,
    SUM(CASE WHEN work_schedule = 'Удаленная работа' THEN 1 ELSE 0 END) AS remote_vacancies,
    SUM(CASE WHEN work_schedule = 'Удаленная работа' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS remote_vacancy_ratio,
    SUM(CASE WHEN work_schedule = 'Удаленная работа' THEN 1 ELSE 0 END) * 1.0 / total_remote AS remote_vacancy_share -- Доля удаленных вакансий в регионе от общего количества
FROM
    vacancy
JOIN
    area USING (area_id)
JOIN
    total_remote_vacancies -- Присоединяем общую сумму удаленных вакансий
GROUP BY
    region_name, total_remote -- Группируем по региону и общему числу удаленных вакансий
ORDER BY 
    remote_vacancy_share DESC

"""

In [986]:
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    
df

,region_name,total_vacancies,remote_vacancies,remote_vacancy_ratio,remote_vacancy_share
0,Санкт-Петербург,11,4,0.363636,0.571429
1,Новосибирская область,8,2,0.250000,0.285714
2,Москва,11,1,0.090909,0.142857


### 3

Отфильтровал нужные вакансии

In [977]:
query = """
SELECT
    area_name, birth_day, compensation, r.currency, strftime('%Y', date()) - strftime('%Y', birth_day) AS age,

    CASE 
        WHEN r.currency = "EUR" THEN compensation / rate
        WHEN r.currency = "RUR" THEN compensation / rate
        WHEN r.currency = "USD" THEN compensation / rate 
        ELSE NULL 
    END AS salary -- Перевел зп в рубли
FROM
    resume r
JOIN
    area USING (area_id),
    currency ON code = r.currency
WHERE
    area_name IN ('Москва', 'Санкт-Петербург') AND -- Резюме из мск и спб
    birth_day IS NOT Null AND -- без др
    disabled = False AND -- не удалено
    is_finished = 1 -- закончено
ORDER BY resume_id
"""

In [978]:
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    
df

,area_name,birth_day,compensation,currency,age,salary
0,Москва,2008-05-06 00:00:00.000000,300.0,EUR,16,25884.383089
1,Санкт-Петербург,2007-05-06 00:00:00.000000,100000.0,RUR,17,100000.000000
2,Москва,1989-05-06 00:00:00.000000,NaN,EUR,35,NaN
3,Москва,1990-05-06 00:00:00.000000,400.0,USD,34,30595.074193
4,Санкт-Петербург,1982-05-06 00:00:00.000000,75000.0,RUR,42,75000.000000
5,Москва,2007-05-06 00:00:00.000000,NaN,EUR,17,NaN
6,Санкт-Петербург,1991-01-01 00:00:00.000000,80000.0,RUR,33,80000.000000
7,Санкт-Петербург,1977-10-12 00:00:00.000000,3500.0,USD,47,267706.899189
8,Москва,1981-05-05 00:00:00.000000,600.0,USD,43,45892.611290
9,Москва,1976-01-01 00:00:00.000000,4000.0,EUR,48,345125.107852


Делаю временную таблицу и обозначаю возраст

In [970]:
query = """
WITH filtered_resumes AS (
SELECT
    resume_id, area_name, birth_day, compensation, r.currency, strftime('%Y', date()) - strftime('%Y', birth_day) AS age,
    CASE 
        WHEN r.currency = "EUR" THEN compensation / rate
        WHEN r.currency = "RUR" THEN compensation / rate
        WHEN r.currency = "USD" THEN compensation / rate 
        ELSE NULL 
    END AS salary -- Перевел зп в рубли
FROM
    resume r
JOIN
    area USING (area_id),
    currency ON code = r.currency
WHERE
    area_name IN ('Москва', 'Санкт-Петербург') AND -- Резюме из мск и спб
    birth_day IS NOT Null AND -- без др
    disabled = False AND -- не удалено
    is_finished = 1 -- закончено
ORDER BY resume_id
)
SELECT resume_id, area_name, salary, age,
    CASE 
        WHEN age <= 17 THEN '17 лет и младше'
        WHEN age BETWEEN 18 AND 24 THEN '18–24'
        WHEN age BETWEEN 25 AND 34 THEN '25–34'
        WHEN age BETWEEN 35 AND 44 THEN '35–44'
        WHEN age BETWEEN 45 AND 54 THEN '45–54'
        ELSE '55 и старше'
        END AS age_group
FROM filtered_resumes
"""

In [972]:
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    
df

,resume_id,area_name,salary,age,age_group
0,1,Москва,25884.383089,16,17 лет и младше
1,2,Санкт-Петербург,100000.000000,17,17 лет и младше
2,4,Москва,NaN,35,35–44
3,6,Москва,30595.074193,34,25–34
4,7,Санкт-Петербург,75000.000000,42,35–44
5,8,Москва,NaN,17,17 лет и младше
6,12,Санкт-Петербург,80000.000000,33,25–34
7,13,Санкт-Петербург,267706.899189,47,45–54
8,16,Москва,45892.611290,43,35–44
9,18,Москва,345125.107852,48,45–54


Делаю перцентили

In [981]:
query = """
WITH
filtered_resumes AS (
SELECT
    area_name, birth_day, compensation, r.currency, strftime('%Y', date()) - strftime('%Y', birth_day) AS age,
    CASE 
        WHEN r.currency = "EUR" THEN compensation / rate
        WHEN r.currency = "RUR" THEN compensation / rate
        WHEN r.currency = "USD" THEN compensation / rate 
        ELSE NULL 
    END AS salary -- Перевел зп в рубли
FROM
    resume r
JOIN
    area USING (area_id),
    currency ON code = r.currency
WHERE
    area_name IN ('Москва', 'Санкт-Петербург') AND -- Резюме из мск и спб
    birth_day IS NOT Null AND -- без др
    disabled = False AND -- не удалено
    is_finished = 1 -- закончено
ORDER BY resume_id
),
grouped_ages AS (
SELECT area_name, salary,
    CASE 
        WHEN age <= 17 THEN '17 лет и младше'
        WHEN age BETWEEN 18 AND 24 THEN '18–24'
        WHEN age BETWEEN 25 AND 34 THEN '25–34'
        WHEN age BETWEEN 35 AND 44 THEN '35–44'
        WHEN age BETWEEN 45 AND 54 THEN '45–54'
        ELSE '55 и старше'
        END AS age_group
FROM filtered_resumes
)

SELECT 
    area_name,
    age_group,
    (COUNT(salary) * 1.0 / COUNT(*)) AS salary_ratio,
    AVG(salary) / 2,
    AVG(salary),
    SUM(salary) * 0.9 
FROM grouped_ages
GROUP BY area_name, age_group
ORDER BY area_name, age_group
"""

In [982]:
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    
df

,area_name,age_group,salary_ratio,AVG(salary) / 2,AVG(salary),SUM(salary) * 0.9
0,Москва,17 лет и младше,0.5,12942.191544,25884.383089,23295.944780
1,Москва,25–34,1.0,15297.537097,30595.074193,27535.566774
2,Москва,35–44,0.5,22946.305645,45892.611290,41303.350161
3,Москва,45–54,1.0,172562.553926,345125.107852,310612.597066
4,Санкт-Петербург,17 лет и младше,1.0,50000.000000,100000.000000,90000.000000
5,Санкт-Петербург,25–34,1.0,40000.000000,80000.000000,72000.000000
6,Санкт-Петербург,35–44,1.0,67159.907967,134319.815933,362663.503020
7,Санкт-Петербург,45–54,1.0,133853.449595,267706.899189,240936.209270
